In [ ]:
# Pump‑stats API workflow
# Этот ноутбук демонстрирует **полный цикл** работы с MEXC‑API через класс `PumpWatcher`, включая:
# 1. очистку таблицы `pump_event_stats`,
# 2. получение последних свечей от API,
# 3. фиксацию новых памп‑ивентов в БД,
# 4. просмотр записей и сводной статистики.


In [14]:

# > Все функции определены прямо в ноутбуке, поэтому ничего импортировать не нужно.
import time, requests, psycopg2, logging
from datetime import datetime, timedelta, timezone
from typing import Optional, Tuple, Dict, List, Callable


In [15]:

# ————————————————————————————————————————————————
# 1) Логирование
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s] %(levelname)s: %(message)s",
    datefmt="%H:%M:%S",
)
logger = logging.getLogger(__name__)


In [54]:

# ————————————————————————————————————————————————
# 2) Параметры подключения к Postgres
DB: Dict[str, str | int] = {
    "dbname": "mexc_statistics_case",
    "user": "postgres",
    "password": "1374fjsney831",
    "host": "localhost",
    "port": 5432,
}

SYMBOL: str = "ALPACA_USDT"
PUMP_THR: float = 1.001     # +7 % к цене 3 ч назад → считаем памп
# ————————————————————————————————————————————————
def _pg():
    """Подключение к PostgreSQL с autocommit."""
    conn = psycopg2.connect(**DB)
    conn.autocommit = True
    return conn

def init_table() -> None:
    """Создаём таблицу, если её ещё нет."""
    ddl = """
    CREATE TABLE IF NOT EXISTS pump_event_stats (
        id            SERIAL PRIMARY KEY,
        coin_symbol   TEXT NOT NULL REFERENCES coins(coin) ON DELETE CASCADE,
        pump_start_ts BIGINT NOT NULL,
        pump_end_ts   BIGINT,
        duration_sec  INTEGER,
        diff_percent  NUMERIC(8,3),
        bucket        VARCHAR(20),
        is_profit     BOOLEAN,
        UNIQUE (coin_symbol, pump_start_ts)
    );"""
    with _pg() as conn, conn.cursor() as cur:
        cur.execute(ddl)

def clear_table(symbol: str) -> None:
    """Удаляет все записи для указанного символа."""
    with _pg() as conn, conn.cursor() as cur:
        cur.execute("DELETE FROM pump_event_stats WHERE coin_symbol=%s", (symbol,))

def upsert_stat(symbol: str, ts_start: int, ts_end: Optional[int],
                diff_pct: Optional[float], bucket: str, is_profit: Optional[bool]) -> None:
    duration = ts_end - ts_start if ts_end else None
    sql = """
    INSERT INTO pump_event_stats
          (coin_symbol, pump_start_ts, pump_end_ts,
           duration_sec, diff_percent, bucket, is_profit)
    VALUES (%s,%s,%s,%s,%s,%s,%s)
    ON CONFLICT (coin_symbol, pump_start_ts)
        DO UPDATE SET
            pump_end_ts   = EXCLUDED.pump_end_ts,
            duration_sec  = EXCLUDED.duration_sec,
            diff_percent  = EXCLUDED.diff_percent,
            bucket        = EXCLUDED.bucket,
            is_profit     = EXCLUDED.is_profit;"""
    with _pg() as conn, conn.cursor() as cur:
        cur.execute(sql, (symbol, ts_start, ts_end, duration,
                          diff_pct, bucket, is_profit))

def stats_last72h(symbol: str) -> Dict[str, int | float]:
    t0 = int((datetime.now(timezone.utc) - timedelta(days=3)).timestamp())
    sql = """
    WITH s AS (
        SELECT * FROM pump_event_stats
        WHERE coin_symbol=%s AND pump_start_ts >= %s
    )
    SELECT
        SUM((bucket='fifteen_minutes')::int) AS fifteen,
        SUM((bucket='three_hours')::int)     AS three_h,
        SUM((bucket='one_day')::int)         AS one_d,
        SUM((bucket='more_one_day')::int)    AS more_d,
        SUM((bucket='in_deal')::int)         AS in_deal,
        COUNT(*)                             AS total,
        SUM((is_profit IS TRUE)::int)        AS wins,
        SUM((is_profit IS FALSE)::int)       AS losses,
        ROUND(COALESCE(SUM(diff_percent)
              FILTER (WHERE is_profit),0),2) AS sum_profit_pct,
        ROUND(COALESCE(SUM(diff_percent)
              FILTER (WHERE is_profit IS FALSE),0),2) AS sum_loss_pct
    FROM s;"""
    with _pg() as conn, conn.cursor() as cur:
        cur.execute(sql, (symbol, t0))
        return dict(zip([d.name for d in cur.description], cur.fetchone()))


In [55]:

# ————————————————————————————————————————————————
# 3) Работа с MEXC API: берём последнюю 15‑мин свечу
def last_candle(symbol: str,
                session: Callable[..., requests.Response] = requests.get,
                interval: str = "Min1") -> Optional[Tuple[int, float]]:
    """
    Возвращает (ts_open_sec, open_price) последней свечи.
    Работает с актуальным endpoint contract.mexc.com и форматом ответа.
    """
    url = f"https://contract.mexc.com/api/v1/contract/kline/{symbol}"
    params = {"interval": interval}          # start/end не нужны
    try:
        resp = session(url, params=params, timeout=10)
        resp.raise_for_status()
        j = resp.json()
        if not j.get("success", True):
            logger.warning("MEXC error: %s", j.get("msg", j.get("code")))
            return None
        # «data» — словарь с полями‑массива­ми. Берём последние значения
        ts_open_sec = int(j["data"]["time"][-1])
        price_open  = float(j["data"]["open"][-1])
        return ts_open_sec, price_open
    except Exception as exc:
        logger.warning("network/json error: %s", exc)
        return None
# ————————————————————————————————————————————————
# 4) Класс PumpWatcher (упрощённая версия)
class PumpWatcher:
    """Отслеживает начало/конец пампов, пишет в БД."""
    def __init__(self, symbol: str, pump_thr: float = PUMP_THR,
                 api_fn: Callable[[str], Optional[Tuple[int, float]]] = last_candle):
        self.symbol = symbol
        self.pump_thr = pump_thr
        self.api_fn = api_fn
        self.history: List[Tuple[int, float]] = []
        self.open_pumps: Dict[int, float] = {}

    @staticmethod
    def _bucket(age: int) -> str:
        if age < 900:
            return "fifteen_minutes"
        if age < 3 * 3600:
            return "three_hours"
        if age < 24 * 3600:
            return "one_day"
        return "more_one_day"

    def step(self) -> None:
        candle = self.api_fn(self.symbol)
        if candle is None:
            return
        ts, price = candle
        self.history.append((ts, price))

        # 1) обнаружить старт пампа
        ts_3h = ts - 3 * 3600
        base_price = next((p for (t, p) in reversed(self.history) if t <= ts_3h), None)
        if base_price and price >= base_price * self.pump_thr:
            if ts not in self.open_pumps:
                self.open_pumps[ts] = price
                logger.info("🚀 START %s price=%.4f", datetime.utcfromtimestamp(ts), price)
                upsert_stat(self.symbol, ts, None, None, "in_deal", None)

        # 2) закрыть пампы?
        to_close = []
        for ts_start, p_start in self.open_pumps.items():
            age = ts - ts_start
            if price <= p_start * 0.93:  # take‑profit
                bucket = self._bucket(age)
                diff = round((price - p_start) / p_start * 100, 3)
                upsert_stat(self.symbol, ts_start, ts, diff, bucket, True)
                logger.info("✅ PROFIT close %dm diff=%s%%", age // 60, diff)
                to_close.append(ts_start)
            elif age > 24 * 3600:        # stop‑loss по времени
                diff = round((price - p_start) / p_start * 100, 3)
                upsert_stat(self.symbol, ts_start, ts, diff, "more_one_day", False)
                logger.info("❌ LOSS close >24h diff=%s%%", diff)
                to_close.append(ts_start)
        for t in to_close:
            del self.open_pumps[t]

print("Функции и класс PumpWatcher готовы!")


Функции и класс PumpWatcher готовы!


In [56]:

## Шаг 1 — подготовить базу: создать таблицу и очистить данные
init_table()
clear_table(SYMBOL)
print(f"Таблица 'pump_event_stats' очищена для {SYMBOL} ✨")


Таблица 'pump_event_stats' очищена для ALPACA_USDT ✨


In [57]:
query = """
    SELECT coin_symbol,
            id,
           to_timestamp(pump_start_ts) AS start_utc,
           to_timestamp(pump_end_ts)   AS end_utc,
           duration_sec,
           diff_percent,
           bucket,
           is_profit
    FROM pump_event_stats
    -- WHERE coin_symbol = %s
    ORDER BY id DESC
    LIMIT 20
"""
with _pg() as conn:                         # берём то же соединение‑хелпер
    df = pd.read_sql_query(query, conn, params=(SYMBOL,))

df     # просто вернуть DataFrame — Jupyter покажет его как таблицу

/tmp/ipykernel_1622/1601844649.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=(SYMBOL,))


,coin_symbol,id,start_utc,end_utc,duration_sec,diff_percent,bucket,is_profit
0,AIL_USDT,109,2025-04-06 10:06:06+00:00,2025-04-06 10:21:12+00:00,906.0,-7.438,fifteen_minutes,True
1,AIL_USDT,108,2025-04-06 09:53:33+00:00,NaT,NaN,NaN,in_deal,None
2,AIL_USDT,107,2025-04-06 09:50:15+00:00,NaT,NaN,NaN,in_deal,None
3,AIL_USDT,106,2025-04-06 05:35:37+00:00,NaT,NaN,NaN,in_deal,None
4,AIL_USDT,105,2025-04-06 04:47:32+00:00,NaT,NaN,NaN,in_deal,None
5,AIL_USDT,104,2025-04-05 20:36:51+00:00,NaT,NaN,NaN,in_deal,None
6,AIL_USDT,103,2025-04-05 20:29:55+00:00,NaT,NaN,NaN,in_deal,None
7,AIL_USDT,102,2025-04-05 18:17:07+00:00,NaT,NaN,NaN,in_deal,None
8,AIL_USDT,101,2025-04-05 15:38:11+00:00,2025-04-05 20:00:33+00:00,15742.0,-16.423,fifteen_minutes,True
9,AIL_USDT,100,2025-04-05 14:32:04+00:00,2025-04-05 20:00:33+00:00,19709.0,-13.468,fifteen_minutes,True


In [58]:
## Шаг 2 — сделать один запрос к API и обработать через PumpWatcher
watcher = PumpWatcher(SYMBOL)
watcher.step()   # Один вызов; можно повторять вручную

## Шаг 3 — посмотреть последние записи в таблице
with _pg() as conn, conn.cursor() as cur:
    cur.execute(
        """
        SELECT id, to_timestamp(pump_start_ts) AS start_utc,
               to_timestamp(pump_end_ts)   AS end_utc,
               duration_sec, diff_percent,
               bucket, is_profit
        FROM pump_event_stats
        WHERE coin_symbol=%s
        ORDER BY id DESC
        LIMIT 10
        """, (SYMBOL,)
    )
    cols = [d.name for d in cur.description]
    for row in cur.fetchall():
        print(dict(zip(cols, row)))


In [59]:

## Шаг 4 — агрегированная статистика за 72 ч
stats = stats_last72h(SYMBOL)
for k, v in stats.items():
    print(f"{k:15} → {v}")


fifteen         → None
three_h         → None
one_d           → None
more_d          → None
in_deal         → None
total           → 0
wins            → None
losses          → None
sum_profit_pct  → 0.00
sum_loss_pct    → 0.00


In [ ]:
## Шаг 5 (длительный) — непрерывный мониторинг
# Запустить эту ячейку, если хочешь собирать данные в реальном времени.
# Нажать `⏹️` (stop) для остановки.
import itertools
try:
    for _ in itertools.count():          # бесконечный цикл
        watcher.step()
        time.sleep(2)              # пауза - несколько секунд
except KeyboardInterrupt:
    print("⏹️  Остановлено пользователем")

In [ ]:
import pandas as pd
pd.read_sql_query(
    "SELECT id, to_timestamp(pump_start_ts) AS start_utc, bucket "
    "FROM pump_event_stats WHERE coin_symbol=%s ORDER BY id DESC LIMIT 5",
    _pg(), params=(SYMBOL,)
)

/tmp/ipykernel_1622/1741641148.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(


,id,start_utc,bucket
